In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import time

import dataset.dataset as dataset
import datasplit.datasplit as datasplit
import model.models as models
import trainer.trainer as trainer
import utils.utils as utils

torch.cuda.device_count()
cuda0 = torch.device('cuda:0')
cuda1 = torch.device('cuda:1')
cuda2 = torch.device('cuda:2')
cuda3 = torch.device('cuda:3')
device = torch.device(cuda0 if torch.cuda.is_available() else "cpu")

### INIT

In [2]:
# transforms
transform = transforms.Compose([
        transforms.ToTensor(),
    ])

# dataset
root = '/Volumes/Macintosh HD/DATASETS/GUITAR-FX/Mono_Random_Bg'
excl_folders = ['NoFX_mono_preprocessed', 'MT2']
spectra_folder= 'mel_22050_1024_512'
proc_settings_csv = 'proc_settings.csv'
max_num_settings=3

dataset = dataset.FxDataset(root=root,
                            excl_folders=excl_folders, 
                            spectra_folder=spectra_folder, 
                            processed_settings_csv=proc_settings_csv,
                            max_num_settings=max_num_settings,
                            transform=transform)
dataset.init_dataset()
# dataset.generate_mel()

# split
# set test_train_split=0.0 and val_train_split=0.0 to test pre-trained model
split = datasplit.DataSplit(dataset, test_train_split=0.8, val_train_split=0.1, shuffle=True)
# loaders
train_loader, val_loader, test_loader = split.get_split(batch_size=100)

In [3]:
print('dataset size: ', len(dataset))
print('train set size: ', len(split.train_sampler))
print('val set size: ', len(split.val_sampler))
print('test set size: ', len(split.test_sampler))

dataset size:  130000
train set size:  93600
val set size:  10400
test set size:  26000


### TRAIN FxNET

In [4]:
# model
fxnet = models.FxNet(n_classes=dataset.num_fx).to(device)
# optimizer
optimizer_fxnet = optim.Adam(fxnet.parameters(), lr=0.001)
# loss function
loss_func_fxnet = nn.CrossEntropyLoss()

In [5]:
print(fxnet)

FxNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=6264, out_features=120, bias=True)
  (batchNorm3): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (batchNorm4): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=60, out_features=13, bias=True)
)


In [6]:
# SAVE
models_folder = '../../saved/models'
model_name = '20201024_fxnet_mono_rnd_bg_best'
results_folder = '../../saved/results'
results_subfolder = '20201024_fxnet_mono_rnd_bg'

In [7]:
# TRAIN and TEST FxNet OVER MULTIPLE EPOCHS
train_set_size = len(split.train_sampler)
val_set_size = len(split.val_sampler)
test_set_size = len(split.test_sampler)

all_train_losses, all_val_losses, all_test_losses = [],[],[]
all_train_correct, all_val_correct, all_test_correct = [],[],[]
all_train_results, all_val_results, all_test_results = [],[],[]

best_val_correct = 0
early_stop_counter = 0

start = time.time()

for epoch in range(100):
    train_loss, train_correct, train_results = trainer.train_fx_net(
        model=fxnet,
        optimizer=optimizer_fxnet, 
        train_loader=train_loader, 
        train_sampler=split.train_sampler, 
        epoch=epoch, 
        device=device
    )
    
    val_loss, val_correct, val_results = trainer.val_fx_net(
        model=fxnet, 
        val_loader=val_loader, 
        val_sampler=split.val_sampler, 
        device=device
    )
    
    test_loss, test_correct, test_results = trainer.test_fx_net(
        model=fxnet, 
        test_loader=test_loader, 
        test_sampler=split.test_sampler, 
        device=device
    )
    # save models
    if val_correct > best_val_correct:
        best_val_correct = val_correct
        torch.save(fxnet, '%s/%s' % (models_folder, model_name))
        early_stop_counter = 0
        print('\n=== saved best model ===\n')
    else:
        early_stop_counter += 1
        
    # append results
    all_train_losses.append(train_loss)
    all_val_losses.append(val_loss)
    all_test_losses.append(test_loss)
    
    all_train_correct.append(train_correct)
    all_val_correct.append(val_correct)
    all_test_correct.append(test_correct)
    
    all_train_results.append(train_results)
    all_val_results.append(val_results)
    all_test_results.append(test_results)

    if early_stop_counter == 15:
        print('\n--- early stop ---\n')
        break

stop = time.time()
print(f"Training time: {stop - start}s")

Train Epoch: 0	[5000/93600 (5%)]	Total Loss: 103.1931	Avg Loss: 0.0206
Train Epoch: 0	[10000/93600 (11%)]	Total Loss: 176.7688	Avg Loss: 0.0177
Train Epoch: 0	[15000/93600 (16%)]	Total Loss: 235.5280	Avg Loss: 0.0157
Train Epoch: 0	[20000/93600 (21%)]	Total Loss: 284.0380	Avg Loss: 0.0142
Train Epoch: 0	[25000/93600 (27%)]	Total Loss: 326.5438	Avg Loss: 0.0131
Train Epoch: 0	[30000/93600 (32%)]	Total Loss: 364.0105	Avg Loss: 0.0121
Train Epoch: 0	[35000/93600 (37%)]	Total Loss: 397.0113	Avg Loss: 0.0113
Train Epoch: 0	[40000/93600 (43%)]	Total Loss: 427.3789	Avg Loss: 0.0107
Train Epoch: 0	[45000/93600 (48%)]	Total Loss: 456.0176	Avg Loss: 0.0101
Train Epoch: 0	[50000/93600 (53%)]	Total Loss: 481.4139	Avg Loss: 0.0096
Train Epoch: 0	[55000/93600 (59%)]	Total Loss: 506.0730	Avg Loss: 0.0092
Train Epoch: 0	[60000/93600 (64%)]	Total Loss: 530.2398	Avg Loss: 0.0088
Train Epoch: 0	[65000/93600 (69%)]	Total Loss: 552.9837	Avg Loss: 0.0085
Train Epoch: 0	[70000/93600 (75%)]	Total Loss: 575.11

In [8]:
# BEST RESULTS
print('Accuracy: ', 100 * max(all_train_correct) / train_set_size)
print('Epoch: ', np.argmax(all_train_correct))
print()
print('Accuracy: ', 100 * max(all_val_correct) / val_set_size)
print('Epoch: ', np.argmax(all_val_correct))
print()
print('Accuracy: ', 100 * max(all_test_correct) / test_set_size)
print('Epoch: ', np.argmax(all_test_correct))
print()

Accuracy:  93.20726495726495
Epoch:  81

Accuracy:  91.32692307692308
Epoch:  67

Accuracy:  90.84615384615384
Epoch:  67



In [9]:
# SAVE RESULTS - all losses, all correct, best results
all_train_losses_npy = np.array(all_train_losses)
all_train_correct_npy = np.array(all_train_correct)
best_train_results_npy = np.array(all_train_results[67])

all_val_losses_npy = np.array(all_val_losses)
all_val_correct_npy = np.array(all_val_correct)
best_val_results_npy = np.array(all_val_results[67])

all_test_losses_npy = np.array(all_test_losses)
all_test_correct_npy = np.array(all_test_correct)
best_test_results_npy = np.array(all_test_results[67])

fx_labels_npy = np.array(list(dataset.fx_to_label.keys()))

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_losses')), arr=all_train_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_train_correct')), arr=all_train_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_train_results')), arr=best_train_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_losses')), arr=all_val_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_val_correct')), arr=all_val_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_val_results')), arr=best_val_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_losses')), arr=all_test_losses_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'all_test_correct')), arr=all_test_correct_npy)
np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'best_test_results')), arr=best_test_results_npy)

np.save(file=('%s/%s/%s' % (results_folder, results_subfolder, 'fx_labels')), arr=fx_labels_npy)